# 2. Kaggle Cat&Dog

## Import libraries

In [1]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch import nn, optim
from torch.nn import functional as F
import torchvision
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from torchvision import transforms

import tqdm
from PIL import Image

## Import Data

In [2]:
base_dir = "E:/data/DogCat/dogs-vs-cats-redux-kernels-edition"
train_dir = base_dir+'/train/train'
test_dir = base_dir + '/test/test'
train_file = os.listdir(train_dir)
test_file = os.listdir(test_dir)

## Preprocess Data

In [3]:
cat_file = [tf for tf in train_file if 'cat' in tf]
dog_file = [tf for tf in train_file if 'dog' in tf]

In [4]:
len(cat_file)

12500

In [5]:
class CatDog(Dataset):
    def __init__(self, file_list, dir, mode='train', transform=None):
        self.file_list = file_list
        self.dir = dir
        self.mode = mode
        self.transform = transform
        if self.mode == 'train':
            if 'dog' in self.file_list[0]:
                self.label = 1
            else:
                self.label = 0

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, index):
        img = Image.open(os.path.join(self.dir+"/"+self.file_list[index]))
        if self.transform:
            img = self.transform(img)
        if self.mode == 'train':
            img = img.numpy()
            return img.astype('float32'), self.label
        else:
            img = img.numpy()
            return img.astype('float32'), self.file_list[index]


transform = transforms.Compose([
    transforms.Resize(256),
    transforms.ColorJitter(),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.Resize(128),
    transforms.ToTensor()
])

In [6]:
cats = CatDog(cat_file, train_dir, transform=transform)
dogs = CatDog(dog_file, train_dir, transform=transform)

In [7]:
cats.__getitem__(index=0)

(array([[[1.        , 0.99607843, 0.99215686, ..., 0.9529412 ,
          0.98039216, 0.98039216],
         [1.        , 0.99607843, 0.99215686, ..., 0.98039216,
          0.99215686, 0.98039216],
         [1.        , 0.99607843, 0.99215686, ..., 0.99215686,
          0.9843137 , 0.9764706 ],
         ...,
         [0.04705882, 0.05490196, 0.07058824, ..., 0.30980393,
          0.31764707, 0.31764707],
         [0.05490196, 0.0627451 , 0.07843138, ..., 0.30588236,
          0.30980393, 0.3137255 ],
         [0.04705882, 0.05490196, 0.07058824, ..., 0.3019608 ,
          0.30588236, 0.30980393]],
 
        [[0.91764706, 0.9137255 , 0.9098039 , ..., 0.79607844,
          0.84705883, 0.87058824],
         [0.91764706, 0.9137255 , 0.9098039 , ..., 0.8392157 ,
          0.8784314 , 0.88235295],
         [0.91764706, 0.9137255 , 0.9098039 , ..., 0.8745098 ,
          0.88235295, 0.8862745 ],
         ...,
         [0.04705882, 0.05490196, 0.0627451 , ..., 0.19607843,
          0.20392157, 0.

In [8]:
catdogs = ConcatDataset([cats, dogs])

dataloader = DataLoader(catdogs, batch_size=32, shuffle=True, num_workers=4)

## Building model

In [9]:
device = 'cuda' if torch.cuda.is_available else 'cpu'
device

'cuda'

In [10]:
model = torchvision.models.densenet121(pretrained=True)

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to C:\Users\WenBi/.cache\torch\hub\checkpoints\densenet121-a639ec97.pth


  0%|          | 0.00/30.8M [00:00<?, ?B/s]

In [11]:
print(model)

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [12]:
num_ = model.classifier.in_features
model.classifier = nn.Sequential(
    nn.Linear(num_, 500),
    nn.Linear(500, 2))
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.002, amsgrad=True)
scheduler = torch.optim.lr_scheduler.MultiStepLR(
    optimizer, milestones=[500, 1000, 1500], gamma=0.5)

## Train

In [ ]:
epochs = 3
itr = 1
p_iter = 200
model.train()
total_loss = 0
loss_list = []
acc_list = []

for epoch in range(epochs):
    for imgs, labels in dataloader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        yhat = model(imgs)
        loss = criterion(yhat,labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        scheduler.step()
        
        if itr%p_iter == 0:
            pred = torch.argmax(yhat,dim=1)
            correct = pred.eq(labels)
            acc = torch.mean(correct.float())
            print('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch+1, epochs, itr, total_loss/p_itr, acc))
            loss_list.append(total_loss/p_itr)
            acc_list.append(acc)
            total_loss = 0
            
        itr += 1
        

plt.plot(loss_list, label='loss')
plt.plot(acc_list, label='accuracy')
plt.legend()
plt.title('training loss and accuracy')
plt.show()

## Test

In [ ]:
filename_pth = 'ckpt_densenet121_catdog.pth'
torch.save(model.state_dict(), filename_pth)

test_transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor()
])

testset = CatDogDataset(test_files, test_dir, mode='test', transform = test_transform)
testloader = DataLoader(testset, batch_size = 32, shuffle=False, num_workers=4)

In [ ]:
model.eval()
fn_list = []
pred_list = []
for x, fn in testloader:
    with torch.no_grad():
        x = x.to(device)
        output = model(x)
        pred = torch.argmax(output, dim=1)
        fn_list += [n[:-4] for n in fn]
        pred_list += [p.item() for p in pred]

submission = pd.DataFrame({"id":fn_list, "label":pred_list})
submission.to_csv('preds_densenet121.csv', index=False)

In [ ]:
samples, _ = iter(testloader).next()
samples = samples.to(device)
fig = plt.figure(figsize=(24, 16))
fig.tight_layout()
output = model(samples[:24])
pred = torch.argmax(output, dim=1)
pred = [p.item() for p in pred]
ad = {0:'cat', 1:'dog'}
for num, sample in enumerate(samples[:24]):
    plt.subplot(4,6,num+1)
    plt.title(ad[pred[num]])
    plt.axis('off')
    sample = sample.cpu().numpy()
    plt.imshow(np.transpose(sample, (1,2,0)))